## Importing dependencies

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import load_model
import os 
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt

In [46]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [47]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Removing unwanted images

In [48]:
data_dir = 'data'

In [55]:
image_extensions = ['jpeg','jpg', 'bmp']

In [56]:
#looping through the data directories to get all the images and then making sure they are in the right format

for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('No issue with image {}'.format(image_path))

No issue with image data/non_potatoes/00IMPORTS-videoSixteenByNine3000.jpg
No issue with image data/non_potatoes/1200px-Marketvegetables.jpg
No issue with image data/non_potatoes/123-fruits-and-vegetables.jpg
No issue with image data/non_potatoes/1260616200_5799399964001_5799397206001-vs.jpg
No issue with image data/non_potatoes/1643560104-b9247ef4-5c68-4cf8-ae35-3268df0d34c4.jpg
No issue with image data/non_potatoes/2-2-2-2foodgroups_vegetables_detailfeature.jpg
No issue with image data/non_potatoes/30-low-maintenance-fruits-vegetables-garden-square.jpg
No issue with image data/non_potatoes/3b360279-8b43-40f3-9b11-604749128187.jpg
No issue with image data/non_potatoes/643188-gettyimages-153946385-ca1ccfaad9be44325afc434b305adc0d.jpg
No issue with image data/non_potatoes/650x350_fun_facts_fruits_and_vegetables_slideshow.jpg
No issue with image data/non_potatoes/a-colorful-selection-of-fruits-and-vegetables.jpg
No issue with image data/non_potatoes/a-spread-of-many-fruits-and-vegetables

No issue with image data/non_potatoes/images36.jpg
No issue with image data/non_potatoes/images37.jpg
No issue with image data/non_potatoes/images39.jpg
No issue with image data/non_potatoes/images44.jpg
No issue with image data/non_potatoes/images45.jpg
No issue with image data/non_potatoes/images46.jpg
No issue with image data/non_potatoes/images48.jpg
No issue with image data/non_potatoes/images49.jpg
No issue with image data/non_potatoes/images50.jpg
No issue with image data/non_potatoes/images51.jpg
No issue with image data/non_potatoes/images52.jpg
No issue with image data/non_potatoes/images53.jpg
No issue with image data/non_potatoes/images55.jpg
No issue with image data/non_potatoes/images56.jpg
No issue with image data/non_potatoes/images57.jpg
No issue with image data/non_potatoes/images58.jpg
No issue with image data/non_potatoes/images59.jpg
No issue with image data/non_potatoes/images60.jpg
No issue with image data/non_potatoes/images61.jpg
No issue with image data/non_po

No issue with image data/potatoes/image19.jpeg
No issue with image data/potatoes/image2.jpeg
No issue with image data/potatoes/image20.jpeg
No issue with image data/potatoes/image21.jpeg
No issue with image data/potatoes/image22.jpeg
No issue with image data/potatoes/image23.jpeg
No issue with image data/potatoes/image24.jpeg
No issue with image data/potatoes/image25.jpeg
No issue with image data/potatoes/image26.jpeg
No issue with image data/potatoes/image27.jpeg
No issue with image data/potatoes/image28.jpeg
No issue with image data/potatoes/image29.jpeg
No issue with image data/potatoes/image3.jpeg
No issue with image data/potatoes/image30.jpeg
No issue with image data/potatoes/image31.jpeg
No issue with image data/potatoes/image32.png
No issue with image data/potatoes/image6.jpeg
No issue with image data/potatoes/image7.jpeg
No issue with image data/potatoes/image8.jpeg
No issue with image data/potatoes/image9.jpeg
No issue with image data/potatoes/images.jpg
No issue with image da

libpng warning: iCCP: known incorrect sRGB profile


No issue with image data/potatoes/new-crop-potatoes.png
No issue with image data/potatoes/otato-yukon-gold-potato-11563027478rl3dnf31ii.png
No issue with image data/potatoes/otatoes-xlarge_trans_NvBQzQNjv4Bq-IWLY18X4-CzgyIcjLEAj0k9u7HhRJvuo-ZLenGRumA.jpg
No issue with image data/potatoes/owz42bqlbr5zqmncrbwqgwdbmr4vwcat1dqxarqszldytpei60ebhdmu6ok7vpjqpvr793lr6liy.png
No issue with image data/potatoes/P14-Lily-Potatoes.jpg
No issue with image data/potatoes/pan20roasted20new20potatoes.jpg
No issue with image data/potatoes/Patates.jpg
No issue with image data/potatoes/png-transparent-potato-vegetable-food-fruit-potato-soup-food-baking-thumbnail.png
No issue with image data/potatoes/pngtree-delicious-potatoes-png-image_5637104.jpg
No issue with image data/potatoes/pngtree-three-oval-potatoes-png-image_5544278.jpg
No issue with image data/potatoes/pngtree-three-oval-yellow-potatoes-png-image_5544276.jpg
No issue with image data/potatoes/potato-11526060705mlvu6hgemy.png
No issue with image d

libpng warning: iCCP: known incorrect sRGB profile


No issue with image data/potatoes/potato_PNG7081.png
No issue with image data/potatoes/potato_PNG7084.png
No issue with image data/potatoes/potato_PNG98084.png


libpng warning: iCCP: known incorrect sRGB profile


No issue with image data/potatoes/purepng.com-potatofapotatofood-cropsvegetablepotatos-1701527311348afxiy.png
No issue with image data/potatoes/quiack-roasted-potatoes-2-500x500.jpg
No issue with image data/potatoes/raw-potato-isolated-white-background-with-clipping-path_88281-1368.jpg
No issue with image data/potatoes/Raw-potatoes-in-burlap-bag-on-transparent-background-PNG.png
No issue with image data/potatoes/raw-potatoes-potato-1296x728-header.jpg
No issue with image data/potatoes/rbank-yukon-gold-potato-luther-burbank-a-potato-food-gold-potatoes-thumbnail.png
No issue with image data/potatoes/realistic-potato-on-transparent-background-260nw-1923444077.jpg
No issue with image data/potatoes/Russet_potato_cultivar_with_sprouts.jpg
No issue with image data/potatoes/types-of-potatoes-different-kinds-of-potatoes-1624896435.jpg


## Loading in data with tensorflow

In [57]:
data = tf.keras.preprocessing.image_dataset_from_directory('data')

Found 400 files belonging to 2 classes.


In [58]:
data_iterator = data.as_numpy_iterator()

In [59]:
data_iterator.next()

InvalidArgumentError: Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]

NameError: name 'batch' is not defined